<a href="https://colab.research.google.com/github/romica44/GlaucomaDetection/blob/main/trainingmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DETECCION TEMPRANA DE GLAUCOMA

In [ ]:
# Librerías necesarias
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# Adecuar las imagenes antes de entrenar el algoritmo para que todas tengan el mismo tamaño y color

from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_training = "http://drive.google.com/drive/folders/1rWPaxRdd0bNlX5-9wI_vGiZb6eCHIE7X?usp=sharing"

path_validation = "http://drive.google.com/drive/folders/1wEg0RnxZvs3cj3hLHCvBqPtU-onyGYUF?usp=sharing"


In [ ]:

# Configurar el generador de datos de imágenes
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalizar los valores de píxeles a 0-1
    validation_split=0.2,  # Separar el 20% de las imágenes para validación
)


In [ ]:

# Cargar las imágenes del directorio 'path' y redimensionar a 256x256 píxeles
train_ds = datagen.flow_from_directory(
    path_training,
    target_size=(256, 256),
    batch_size=32,
    subset='training',
)


In [ ]:

# Cargar las imágenes del directorio 'path' y redimensionar a 256x256 píxeles
val_ds = datagen.flow_from_directory(
    path_validation,
    target_size=(256, 256),
    batch_size=32,
    subset='validation',
)



In [ ]:
# Se carga el conjunto de datos y se divide en un conjunto de entrenamiento y otro de prueba

# import matplotlib.pyplot as plt
# from matplotlib.image import imread

# # Ruta de la imagen
# img_path = "/content/drive/My Drive/Glaucoma_Detection/dataset/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Train/Glaucoma_Positive/036.jpg"

# # Leer la imagen
# img = imread(img_path)

# # Mostrar la imagen
# plt.imshow(img)
# plt.axis('off')
# plt.show()


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_training,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=32)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_validation,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(256, 256),
    batch_size=32)



In [ ]:
# Preprocesamiento de datos: En este ejemplo, normalizamos los píxeles de las imágenes y las escalamos a un rango de 0 a 1:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(
    1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
# Modelo de aprendizaje automático elegico CNN (red neuronal convolucional)
model = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])



In [ ]:
# Entrenar el modelo: Utilizamos la función compile para configurar el modelo con una función de pérdida, un optimizador y una métrica de evaluación:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])



In [ ]:

# A continuación, entrenamos el modelo utilizando el conjunto de datos de entrenamiento:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


In [ ]:
# Evaluar el modelo: Finalmente, podemos evaluar la precisión del modelo utilizando el conjunto de datos de prueba:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "path/to/dataset",
    validation_split=None,
    subset="testing",
    seed=123,
    image_size=(256, 256),
    batch_size=32)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

